In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('dataset_fix.csv')

In [3]:
data.head()

,Unnamed: 0,Unnamed: 0.1,conversation_id,date,tweet,tweetbersih,translated,sentiment,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,0,0,1.41E+018,2021-06-24 8:41:03,Di gua udh ga ada sawah digusur sentraland se...,Di gua sudah tidak ada sawah digusur sentralan...,In the cave there are no rice fields evicted b...,negative,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,1.41E+018,2021-06-22 1:05:52,teriak harga properti mahal akhirnya sawah d...,teriak harga properti mahal akhirnya sawah di...,"shouted expensive property prices, finally the...",negative,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,1.41E+018,2021-06-18 8:41:56,Gunungnya udah dikeruk Sawah udah jadi pabrik...,Gunungnya udah dikeruk Sawah udah jadi pabrik...,The mountain has been dredged The rice field h...,negative,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,1.41E+018,2021-06-17 20:55:28,Semua sawah akan digusur dinggo mbangun dalan ...,Semua sawah akan digusur membangun jalan tol p...,All rice fields will be evicted so that they c...,neutral,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,1.41E+018,2021-06-17 19:41:33,Terpikir jg klo fenomena skrg proyek bangunan ...,Terpikir juga kalau fenomena sekarang proyek b...,It also occurred to me that the phenomenon now...,neutral,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
data.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace = True)

In [5]:
data.drop(['date', 'conversation_id', 'tweetbersih', 'translated'], axis = 1, inplace = True)

In [6]:
data.drop(['Unnamed: 8', 'Unnamed: 9', 'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20'], axis = 1, inplace = True)

In [7]:
data.head()

,tweet,sentiment
0,Di gua udh ga ada sawah digusur sentraland se...,negative
1,teriak harga properti mahal akhirnya sawah d...,negative
2,Gunungnya udah dikeruk Sawah udah jadi pabrik...,negative
3,Semua sawah akan digusur dinggo mbangun dalan ...,neutral
4,Terpikir jg klo fenomena skrg proyek bangunan ...,neutral


# Feature Engineering 

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(data.tweet).toarray()

In [36]:
X.shape

(2873, 11315)

In [37]:
cols_num = X.shape[1]
cols_num

11315

In [38]:
train_data = X[:2442]
test_data = X[2442:]
target = data['sentiment'][:2442]

train_data.shape, target.shape, test_data.shape

((2442, 11315), (2442,), (431, 11315))

In [39]:
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

In [40]:
train_df.isnull().sum().any(), test_df.isnull().sum().any()

(False, False)

In [41]:
test_df.head(2)

,0,1,2,3,4,5,6,7,8,9,...,11305,11306,11307,11308,11309,11310,11311,11312,11313,11314
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
# Extract the index where variance is 0.0 in train data
cnt_train = []
for i in range(0,cols_num) :
    if train_df[i].nunique() == 1 :
        cnt_train.append(i)
len(cnt_train)

1153

In [43]:
# Remove columns with index from cnt_train
train_df.drop(cnt_train, axis=1, inplace=True)
test_df.drop(cnt_train, axis=1, inplace=True)
train_df.shape, test_df.shape

((2442, 10162), (431, 10162))

In [44]:
X = train_df.values
X_test = test_df.values

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, target, test_size = 0.3, random_state = 0)

In [46]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
logreg = LogisticRegression(random_state = 0, class_weight='weight')
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_val)
score_logreg = f1_score(y_val, y_pred_logreg)
print(score_logreg)

ValueError: Input contains NaN

In [10]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.15, random_state=75)

In [14]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

gnb = GaussianNB()
gnb.fit(train, test)
print(cross_val_score(gnb, train, test, cv=10).mean())

ValueError: could not convert string to float: 'Buka sawah baru  ada tanah sawah asli jadi proyek jalan tol  pabrik  mall dll'